In [18]:
!pip install fastapi uvicorn nest-asyncio pyngrok itsdangerous python-multipart email-validator passlib


In [19]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import FileResponse
from pyngrok import ngrok
import os
from itsdangerous import URLSafeTimedSerializer
import nest_asyncio
import uvicorn


In [20]:
nest_asyncio.apply()


In [21]:
UPLOAD_DIR = "/content/uploads"
os.makedirs(UPLOAD_DIR, exist_ok=True)

SECRET_KEY = "mihika_secure_key"
serializer = URLSafeTimedSerializer(SECRET_KEY)


In [22]:
app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello, Mihika!"}

@app.post("/ops/upload-file")
async def upload_file(file: UploadFile = File(...)):
    allowed_types = [
        'application/vnd.openxmlformats-officedocument.presentationml.presentation',
        'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
        'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
    ]
    if file.content_type not in allowed_types:
        raise HTTPException(status_code=400, detail="File type not allowed!")

    file_location = os.path.join(UPLOAD_DIR, file.filename)
    with open(file_location, "wb") as buffer:
        buffer.write(await file.read())

    return {"message": "File uploaded successfully!"}

@app.get("/client/download-link/{filename}")
def generate_download_link(filename: str):
    token = serializer.dumps(filename)
    download_url = f"{public_url}/client/download-file/{token}"
    return {"download-link": download_url, "message": "success"}

@app.get("/client/download-file/{token}")
def download_file(token: str):
    try:
        filename = serializer.loads(token, max_age=600)
    except:
        raise HTTPException(status_code=400, detail="Invalid or expired link!")

    file_path = os.path.join(UPLOAD_DIR, filename)
    if not os.path.exists(file_path):
        raise HTTPException(status_code=404, detail="File not found!")

    return FileResponse(file_path, media_type='application/octet-stream', filename=filename)


In [26]:
pip install pyngrok

In [29]:
from pyngrok import ngrok

# Set your authtoken here
ngrok.set_auth_token("2zLp6KBb6YHBGDzLnqtABUyfT4U_i7Wka3Tcf5bAX8en9cu1")


In [30]:
public_url = ngrok.connect(8000)
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://e5d8-34-63-238-224.ngrok-free.app" -> "http://localhost:8000"


In [31]:
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [879]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [879]
